# `Dataset`

> A python list like object that contains your evaluation data.

In [2]:
# | default_exp dataset

In [3]:
# | hide

from unittest.mock import MagicMock

In [4]:
# | export
import typing as t

from fastcore.utils import patch

from ragas_annotator.model.notion_model import NotionModel
from ragas_annotator.backends.notion_backend import NotionBackend

In [29]:
# | export
NotionModelType = t.TypeVar("NotionModelType", bound=NotionModel)


class Dataset(t.Generic[NotionModelType]):
    """A list-like interface for managing NotionModel instances in a Notion database."""

    def __init__(
        self,
        name: str,
        model: t.Type[NotionModel],
        database_id: str,
        notion_backend: NotionBackend,
    ):
        self.name = name
        self.model = model
        self.database_id = database_id
        self._notion_backend = notion_backend
        self._entries: t.List[NotionModelType] = []

    def __getitem__(
        self, key: t.Union[int, slice]
    ) -> t.Union[NotionModelType, "Dataset[NotionModelType]"]:
        """Get an entry by index or slice."""
        if isinstance(key, slice):
            new_dataset = type(self)(
                name=self.name,
                model=self.model,
                database_id=self.database_id,
                notion_backend=self._notion_backend,
            )
            new_dataset._entries = self._entries[key]
            return new_dataset
        else:
            return self._entries[key]

    def __setitem__(self, index: int, entry: NotionModelType) -> None:
        """Update an entry at the given index and sync to Notion."""
        if not isinstance(entry, self.model):
            raise TypeError(f"Entry must be an instance of {self.model.__name__}")

        # Get existing entry to get Notion page ID
        existing = self._entries[index]
        if not hasattr(existing, "_page_id"):
            raise ValueError("Existing entry has no page_id")

        # Update in Notion
        assert (
            existing._page_id is not None
        )  # mypy fails to infer that we check for it above
        response = self._notion_backend.update_page(
            page_id=existing._page_id, properties=entry.to_notion()["properties"]
        )

        # Update local cache with response data
        self._entries[index] = self.model.from_notion(response)

    def __repr__(self) -> str:
        return f"Dataset(name={self.name}, model={self.model.__name__}, len={len(self)})"

    def __len__(self) -> int:
        return len(self._entries)

    def __iter__(self) -> t.Iterator[NotionModelType]:
        return iter(self._entries)

In [23]:
#| hide
import ragas_annotator.model.notion_typing as nmt

In [30]:
#| hide
example_notion_backend = MagicMock(spec=NotionBackend)
test_database_id = "test_database_id"

# test model
class TestModel(NotionModel):
    id: int = nmt.ID()
    name: str = nmt.Title()
    description: str = nmt.Text()

# create a new dataset and slice it
dataset = Dataset(
    name="TestModel",
    model=TestModel,
    database_id=test_database_id,
    notion_backend=example_notion_backend,
)

# fill the _entries with mock data
for i in range(10):
    dataset._entries.append(TestModel(name=f"test_{i}", description=f"test description {i}"))

In [31]:
#| hide
assert len(dataset) == 10

In [32]:
dataset[0:5]

Dataset(name=TestModel, model=TestModel, len=5)

In [6]:
test_database_id = "1b25d9bf-94ff-81b8-ad75-fe31c9e3b2d2"
root_page_id = "1b05d9bf94ff8092b52ae8d676e6abf2"
example_notion_backend = NotionBackend(root_page_id=root_page_id)

In [8]:
# test model
class TestModel(NotionModel):
    id: int = nmt.ID()
    name: str = nmt.Title()
    description: str = nmt.Text()


test_model = TestModel(name="test", description="test description")
test_model

TestModel(name='test' description='test description')

In [9]:
dataset = Dataset(
    name="TestModel",
    model=TestModel,
    database_id=test_database_id,
    notion_backend=example_notion_backend,
)

In [10]:
# | export
@patch
def append(self: Dataset, entry: NotionModelType) -> None:
    """Add a new entry to the dataset and sync to Notion."""
    # if not isinstance(entry, self.model):
    #     raise TypeError(f"Entry must be an instance of {self.model.__name__}")

    # Create in Notion and get response
    response = self._notion_backend.create_page_in_database(
        database_id=self.database_id, properties=entry.to_notion()["properties"]
    )

    # Update entry with Notion data (like ID)
    updated_entry = self.model.from_notion(response)
    self._entries.append(updated_entry)

In [11]:
test_model._page_id

In [12]:
dataset.append(test_model)
len(dataset)

1

In [13]:
test_model.id

In [14]:
# | export
@patch
def pop(self: Dataset, index: int = -1) -> NotionModelType:
    """Remove and return entry at index, sync deletion to Notion."""
    entry = self._entries[index]
    if not hasattr(entry, "_page_id"):
        raise ValueError("Entry has no page_id")

    # Archive in Notion (soft delete)
    assert entry._page_id is not None  # mypy fails to infer that we check for it above
    self._notion_backend.update_page(page_id=entry._page_id, archived=True)

    # Remove from local cache
    return self._entries.pop(index)

In [15]:
dataset.pop()
len(dataset)

0

In [16]:
# | export
@patch
def load(self: Dataset) -> None:
    """Load all entries from the Notion database."""
    # Query the database
    response = self._notion_backend.query_database(
        database_id=self.database_id, archived=False
    )

    # Clear existing entries
    self._entries.clear()

    # Convert results to model instances
    for page in response.get("results", []):
        entry = self.model.from_notion(page)
        self._entries.append(entry)

In [17]:
for i in range(3):
    dataset.append(test_model)
len(dataset)

3

In [18]:
# create a new instance of the dataset
dataset = Dataset(
    name="TestModel",
    model=TestModel,
    database_id=test_database_id,
    notion_backend=example_notion_backend,
)
len(dataset)

0

In [19]:
dataset.load()
len(dataset)

17

In [20]:
# | export
@patch
def get(self: Dataset, id: int) -> t.Optional[NotionModelType]:
    """Get an entry by ID."""
    if not self._notion_backend:
        return None

    # Query the database for the specific ID
    response = self._notion_backend.query_database(
        database_id=self.database_id,
        filter={"property": "id", "unique_id": {"equals": id}},
    )

    if not response.get("results"):
        return None

    return self.model.from_notion(response["results"][0])

In [21]:
test_model = dataset.get(15)
test_model

TestModel(id=15 name='test' description='updated description')

In [22]:
# | export
@patch
def save(self: Dataset, item: NotionModelType) -> None:
    """Save changes to an item to Notion."""
    if not isinstance(item, self.model):
        raise TypeError(f"Item must be an instance of {self.model.__name__}")

    if not hasattr(item, "_page_id"):
        raise ValueError("Item has no page_id")

    # Update in Notion
    assert item._page_id is not None  # mypy fails to infer that we check for it above
    response = self._notion_backend.update_page(
        page_id=item._page_id, properties=item.to_notion()["properties"]
    )

    # Update local cache
    for i, existing in enumerate(self._entries):
        if existing._page_id == item._page_id:
            self._entries[i] = self.model.from_notion(response)
            break

In [23]:
test_model.description = "updated description"
dataset.save(test_model)

In [24]:
dataset.get(15)

TestModel(id=15 name='test' description='updated description')

## [DELETE THIS] Comparsion View for SuperMe

I'm going to build a comparison view for SuperMe.
- get exp 1 data
- get exp 2 data
- make comparison view with grouped by id


In [25]:
# configs
# https://www.notion.so/ragas/Ragas-Dashboard-1a65d9bf94ff8061a82dd8dc31d69949?pvs=4
root_page_id = "1a65d9bf94ff8061a82dd8dc31d69949"
# https://www.notion.so/ragas/v=1aa5d9bf94ff817594b4000c6f22560c&pvs=4
exp_1_database_id = "1aa5d9bf94ff81468fe8ecf15d3cf8c2"
# https://www.notion.so/ragas/1aa5d9bf94ff81278818d8f1d7af1f16?v=1aa5d9bf94ff812b83ca000ceba9187c&pvs=4
exp_2_database_id = "1aa5d9bf94ff81278818d8f1d7af1f16"


super_me_notion_backend = NotionBackend(root_page_id=root_page_id)

In [26]:
# exp 1 model
class ExperimentModel1(NotionModel):
    id: int = nmt.ID()
    query: str = nmt.Title()
    persona: str = nmt.Select()
    response: str = nmt.Text()
    grade: str = nmt.Select(required=False)
    grading_notes: str = nmt.Text(required=False)
    criteria: str = nmt.Text(required=False)
    personalisation: str = nmt.Select(required=False)
    citation_validity: str = nmt.Select(required=False)
    citation_validity_notes: str = nmt.Text(required=False)


# exp 2 model

In [27]:
exp_1 = Dataset(
    name="experiment_one",
    model=ExperimentModel1,
    database_id=exp_1_database_id,
    notion_backend=super_me_notion_backend,
)
exp_1.load()
len(exp_1)

30

In [28]:
eg = exp_1[10]
print(eg.query)
print(eg.criteria)

List all the industries you have worked for

         - Mention all industries worked in, including Software Development and Technology, E-commerce, Education and E-learning, Media and Publication, and Open Source Projects. 
         - Include specific companies like Robusta Dev, FrontRow, Spoyl, Pedagoge, GKToday.in, and AspecScire. 
         - Ensure no industries are omitted from the list provided.


In [29]:
class ExperimentModel2(NotionModel):
    id: int = nmt.ID()
    query: str = nmt.Title()
    persona: str = nmt.Select()
    response: str = nmt.Text()
    grade: str = nmt.Select(required=False)
    reason: str = nmt.Text(required=False)
    criteria: str = nmt.Text(required=False)
    annotator_grade: str = nmt.Select(required=False)
    annotator_reason: str = nmt.Text(required=False)

In [30]:
exp_2 = Dataset(
    name="experiment_two",
    model=ExperimentModel2,
    database_id=exp_2_database_id,
    notion_backend=super_me_notion_backend,
)
exp_2.load()
len(exp_2)

30

In [31]:
eg = exp_2[10]
print(eg.query)
print(eg.criteria)

what is the difference between ragas and deepeval


         - Correctly explain what Ragas is and its role in evaluating LLM applications.
         - Acknowledge the lack of information on Deepeval but suggest a willingness to provide more details if context is offered.
         - Ensure any citations provided are correct and relevant to the discussion.


In [32]:
from notion_annotator_nbdev.model.notion_model import NotionModel

In [33]:
NotionModel._fields

{}

In [35]:
# combine both properties
class CombinedModel(NotionModel):
    id_str: str = nmt.Text()


new_notion_model = CombinedModel
for field in ExperimentModel1._fields.keys():
    new_notion_model._fields[field] = ExperimentModel1._fields[field]
for field in ExperimentModel2._fields.keys():
    if field not in new_notion_model._fields:
        new_notion_model._fields[field] = ExperimentModel2._fields[field]

len(new_notion_model._fields)

14

In [36]:
properties = {}
for field in new_notion_model._fields.keys():
    properties.update(new_notion_model._fields[field]._to_notion_property())

properties

{'id_str': {'type': 'rich_text', 'rich_text': {}},
 'id': {'type': 'unique_id', 'unique_id': {'prefix': None}},
 'query': {'type': 'title', 'title': {}},
 'persona': {'type': 'select', 'select': {}},
 'response': {'type': 'rich_text', 'rich_text': {}},
 'grade': {'type': 'select', 'select': {}},
 'grading_notes': {'type': 'rich_text', 'rich_text': {}},
 'criteria': {'type': 'rich_text', 'rich_text': {}},
 'personalisation': {'type': 'select', 'select': {}},
 'citation_validity': {'type': 'select', 'select': {}},
 'citation_validity_notes': {'type': 'rich_text', 'rich_text': {}},
 'reason': {'type': 'rich_text', 'rich_text': {}},
 'annotator_grade': {'type': 'select', 'select': {}},
 'annotator_reason': {'type': 'rich_text', 'rich_text': {}}}

In [37]:
# https://www.notion.so/ragas/Comparisons-1a65d9bf94ff81a582fff4100d52280d?pvs=4
comparison_page_id = "1a65d9bf94ff81a582fff4100d52280d"

# create a comparison view
comparison_database_id = super_me_notion_backend.create_new_database(
    parent_page_id=comparison_page_id,
    title="experiment_one and experiment_two",
    properties=properties,
)

In [38]:
# group together by query
grouped = []
for eg1 in exp_1:
    for eg2 in exp_2:
        if eg1.query == eg2.query:
            grouped.append((eg1, eg2))

len(grouped)

30

In [39]:
grouped[0][0].query, grouped[0][1].query

('How to build a career in datascience?',
 'How to build a career in datascience?')

In [40]:
comparison_db = Dataset(
    name="experiment_one and experiment_two",
    model=CombinedModel,
    database_id=comparison_database_id,
    notion_backend=super_me_notion_backend,
)
comparison_db.load()
len(comparison_db)

0

In [45]:
eg1_fields = {}
eg2_fields = {}
eg1, eg2 = grouped[0]
for field in eg1._fields.keys():
    eg1_fields[field] = eg1.__getattribute__(field)
for field in eg2._fields.keys():
    eg2_fields[field] = eg2.__getattribute__(field)

combined_fields = (eg1_fields, eg2_fields)

In [46]:
comparison_db.append(CombinedModel(id_str="1", **combined_fields[0]))
comparison_db.append(CombinedModel(id_str="1", **combined_fields[1]))

In [ ]:
from tqdm import tqdm

In [51]:
for i, (eg1, eg2) in enumerate(tqdm(grouped)):
    # combine the fields
    eg1_fields = {}
    for field in eg1._fields.keys():
        eg1_fields[field] = eg1.__getattribute__(field)

    comparison_db.append(CombinedModel(id_str=str(i), **eg1_fields))

    eg2_fields = {}
    for field in eg2._fields.keys():
        eg2_fields[field] = eg2.__getattribute__(field)
    comparison_db.append(CombinedModel(id_str=str(i), **eg2_fields))

100%|██████████| 30/30 [02:03<00:00,  4.11s/it]
